![Piggy bank](piggy_bank.jpg)

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two-year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to ensure it conforms to the specific structure and data types that they specify so that they can then use the cleaned data you provide to set up a PostgreSQL database, which will store this campaign's data and allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split the data, saving three final csv files. Specifically, the three files should have the names and contents as outlined below:

## `client.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `age` | `integer` | Client's age in years | N/A |
| `job` | `object` | Client's type of job | Change `"."` to `"_"` |
| `marital` | `object` | Client's marital status | N/A |
| `education` | `object` | Client's level of education | Change `"."` to `"_"` and `"unknown"` to `np.NaN` |
| `credit_default` | `bool` | Whether the client's credit is in default | Convert to `boolean` data type:<br> `1` if `"yes"`, otherwise `0` |
| `mortgage` | `bool` | Whether the client has an existing mortgage (housing loan) | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0` |

<br>

## `campaign.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | N/A |
| `contact_duration` | `integer` | Last contact duration in seconds | N/A |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | N/A |
| `previous_outcome` | `bool` | Outcome of the previous campaign | Convert to boolean data type:<br> `1` if `"success"`, otherwise `0`. |
| `campaign_outcome` | `bool` | Outcome of the current campaign | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0`. |
| `last_contact_date` | `datetime` | Last date the client was contacted | Create from a combination of `day`, `month`, and a newly created `year` column (which should have a value of `2022`); <br> **Format =** `"YYYY-MM-DD"` |

<br>

## `economics.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | N/A |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three-month rate (daily indicator) | N/A |

In [46]:
import pandas as pd
import numpy as np

# Start coding here...

# Read the CSV file
df = pd.read_csv("bank_marketing.csv")


# Replace '.' with '_' in the 'education' column
df['education']=df['education'].str.replace('.','_')
# Replace '.' with '_' in the 'job' column
df['job']=df['job'].str.replace('.','_')

# Replace "unknown" with np.NaN in the "education" column
df['education']=df['education'].replace("unknown", np.NaN)


# Mapping for campaign_outcome
df['campaign_outcome']=df['campaign_outcome'].map({'yes': True, 'success': True, 'no': False, 'failure': False, 'nonexistent': False})
# Mapping for previous_outcome
df['previous_outcome']=df['previous_outcome'].map({'yes': True, 'success': True, 'no': False, 'failure': False, 'nonexistent': False})
# Mapping for mortgage
df['mortgage']=df['mortgage'].map({'yes': 1, 'no': 0, 'unknown': 0})
# Mapping for credit_default
df['credit_default']=df['credit_default'].map({'yes': 1, 'no': 0, 'unknown': 0})
# Convert the columns to boolean type
df['campaign_outcome']=df['campaign_outcome'].astype(bool)
df['previous_outcome']=df['previous_outcome'].astype(bool)
df['credit_default']=df['credit_default'].astype(bool)
df['mortgage']=df['mortgage'].astype(bool)

# Ensure the day and month columns are strings
df['day']=df['day'].astype(str)
df['month']=df['month'].astype(str)
df['year'] = '2022'
# Combine year, month, and day into a new column for date
df['last_contact_date']=df['year']+'-'+df['month']+'-'+df['day']
# Convert 'last_contact_date' to datetime
df['last_contact_date'] = pd.to_datetime(df['last_contact_date'], format='%Y-%b-%d')
# Drop the 'month', 'year', and 'day' columns
df.drop(['month','year','day'],axis=1,inplace=True)

# Define column names for each part
client_column=['client_id','age','job','marital','education','credit_default','mortgage']
campaign_column=['client_id','number_contacts','contact_duration','previous_campaign_contacts','previous_outcome','campaign_outcome','last_contact_date']
economics_column=['client_id','cons_price_idx','euribor_three_months']
# Split data into three parts based on the specified columns
client=df[client_column]
campaign=df[campaign_column]
economics=df[economics_column]
# Save each part to a new CSV file
client.to_csv('client.csv', index=False)
campaign.to_csv('campaign.csv', index=False)
economics.to_csv('economics.csv', index=False)

In [47]:
df = pd.read_csv("bank_marketing.csv")

for col in ["credit_default", "mortgage", "previous_outcome", "campaign_outcome"]:
    print(col)
    print("--------------")
    print(df[col].value_counts())

credit_default
--------------
no         32588
unknown     8597
yes            3
Name: credit_default, dtype: int64
mortgage
--------------
yes        21576
no         18622
unknown      990
Name: mortgage, dtype: int64
previous_outcome
--------------
nonexistent    35563
failure         4252
success         1373
Name: previous_outcome, dtype: int64
campaign_outcome
--------------
no     36548
yes     4640
Name: campaign_outcome, dtype: int64
